In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorly
!pip install pmdarima
%cd /content/drive/My Drive/Colab Notebooks/Group30_Final_Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1onvLc9hmQDF89OPg7Mt7Gu3hNH54oJiI/Group30_Final_Project


In [3]:
from input.M4DataSet.transformed_data.transform_m4_monthly import m4_monthly

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

In [0]:
import numpy as np
m4_monthly = m4_monthly[np.random.randint(0, (m4_monthly.shape[0] -1 ), 
                                              size=(m4_monthly.shape[0] // 100),
                                              dtype='l')]

In [0]:
m4_monthly.shape

(480, 42)

In [0]:
import numpy as np

from BHT_ARIMA import BHTARIMA
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_BHTARIMA(data):


      p = 3 # p-order
      d = 2 # d-order
      q = 1 # q-order
      taus = [480,20] # MDT-rank
      Rs = [5,5] # tucker decomposition ranks
      k =  15 # iterations
      tol = 0.001 # stop criterion
      Us_mode = 3 # orthogonality mode
      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      for i in range(T_test):

        ts = data[..., i:T-T_test+i].copy()
        n_round += 1
        model = BHTARIMA(ts, p, d, q, taus, Rs, k, tol, verbose=0, Us_mode=Us_mode)
        result, _ = model.run()
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [0]:
BHTARIMA_m4_monthly = run_BHTARIMA(m4_monthly)
BHTARIMA_m4_monthly

{'acc': 0.9038664062021555, 'nrmse': 0.3189549548625747}

In [5]:
import numpy as np
import pmdarima as pm
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_ARIMA(data):

      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        y = data[..., i:T-T_test+i].copy()
        n_round += 1

        for j in range(y.shape[0]):
          
          model = pm.AutoARIMA(seasonal=True,suppress_warnings=True,trace=False,error_action="ignore")

          result = model.fit_predict(y[j], n_periods=1)
          result_full[j, i] = result[..., -1]


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
ARIMA_m4_monthly = run_ARIMA(m4_monthly)
ARIMA_m4_monthly

{'acc': 0.9369759355763357, 'nrmse': 0.2643128608570179}

In [0]:
import numpy as np
import pandas as pd

from fbprophet import Prophet
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_Prophet(data):


      testsize = 0.1
      start = '2017-3'
      freq = 'D'


      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      time_st = pd.period_range(start=start, periods=T, freq=freq).to_timestamp()


      for i in range(T_test):

        y = data[..., i:T-T_test+i+1].copy()
        n_round += 1
        ds = time_st[i:T-T_test+i+1].copy()
        


        for j in range(y.shape[0]):


          DF_ts = pd.DataFrame()
          DF_ts['ds'] = ds.copy()
          DF_ts['y'] = y[j].copy()



          model = Prophet(yearly_seasonality=True,daily_seasonality=True,weekly_seasonality=True,n_changepoints=8)
          model.fit(DF_ts[i:T-T_test+i])

          result = model.predict(DF_ts[['ds']].iloc[[-1]])['yhat'][0]
          result_full[j, i] = result


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)
 

In [15]:
Prophet_m4_monthly = run_Prophet(m4_monthly)
Prophet_m4_monthly

{'acc': 0.9358771812029935, 'nrmse': 0.2629444791687168}

In [0]:
import numpy as np

from statsmodels.tsa.api import VAR
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_VAR(data):
  
      testsize = 0.1
      p = 1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        ts = (data[..., i:T-T_test+i].copy())
        n_round += 1
        model = VAR(ts)
        result = model.fit(p).forecast(ts, 1)
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [14]:
VAR_m4_monthly = run_VAR(m4_monthly)
VAR_m4_monthly

{'acc': 0.5336271504835156, 'nrmse': 0.8017086393808632}

In [0]:
import numpy as np

import xgboost as xgb
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_XGB(data, test_size):

      T_test = test_size

      result_full = np.zeros([data.shape[0], T_test])
      parameters = {'max_depth':(3,4,5,6,7,8,9,10), 
              'subsample':[0.5,0.6, 0.7, 0.8, 0.9],
               'lambda' :[0, 0.25, 0.5, 0.75, 1]}
      clf = GridSearchCV(XGBRegressor(objective ='reg:squarederror'), parameters, verbose = False, cv = 3)
      X = data[..., :-T_test-1]
      y = data[..., -T_test-1]
      clf.fit(X, y)
      for i in range(T_test):
        X_test = data[..., i+1:-T_test+i]
        result = clf.predict(X_test)
        result_full[..., i] = result

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [12]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore")

XGB_m4_monthly = run_XGB(m4_monthly, test_size=m4_monthly.shape[1] // 10)
XGB_m4_monthly

{'acc': 0.9221197507332357, 'nrmse': 0.2518380224383622}